In [12]:
# 숫자 생성 인공지능 개발 원리

# GAN(Generative adversarial network, 갠 or 간) - 적대 생성 신경망
# 생성자와 판별자는 서로를 이기기 위해 학습하기 시작한다. 
# 그러면 생성자는 판별자가 진짜/가짜를 구별 못하도록 진짜같은 것을 만든다
# 두 개의 신경망을 이용해 새로운 그림을 생성하는 기법이 적대적 생성 신경망, GAN이다.

from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras.layers.activation import LeakyReLU # 강좌 기준으론 from keras.layers.advacned_activations import LeakyReLU 였다.
from keras.datasets import mnist
from keras.optimizers import Adam

# 모델 학습을 시각적으로 보여주는 라이브러리. 
# tqdm은 진행을 나타네는 아랍어(taqadum)에서 딴 이름
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# 데이터 불러오기

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# mnist 데이터셋의 그림은 0-255까지의 숫자로 이루어짐. 그 중간값인 127.5를 빼고 다시 그것으로 나누면
# 0은 -1로, 255는 1로 바뀌며 그 중간 값 또한 비율에 따라 축소됨
x_test = (x_test.astype(np.float32) - 127.5)/127.5 

# 28x28 형태인 데이터를 1열로 나타내기위해 데이터의 형태를 바꿈
mnist_data = x_test.reshape(10000, 784)
print(mnist_data.shape)
len(mnist_data)

11490434/11490434 [==============================] - 1s 0us/step
(10000, 784)


10000

In [10]:
# 생성자 신경망 만들기
# 생성자 신경망에 아무런 의미가 없는 숫자(노이즈값)을 입력하면 그럴듯한 숫자 이미지를 만들어낸다.
# 하지만 지금 만든느 생성자 신경망은 아직 학습되지 않아서 적대적 생성 신경망(GAN)을 통해 판별자를 속일 수 있도록 학습시킬 예정

from email import utils


def create_generator():
    generator = Sequential()
    # 입력하는 값은 100으로 하고(의미있는 숫자는 아님), 이 백개의 픽세은 노이즈 값으로 100개의 픽셁밧이 랜덤하게 생성된다.
    # 생성자는 무에서 유를 창조하지 않으므로 이런 노이즈값을 줘야한다.
    # 생성자에게 아무것도 아닌 어떤 데이터를 주면 그것을 특정한 숫자의 모습으로 탈바꿈시킴
    # 이 신경망의 첫 번째 층은 256개의 노드로 구성됨
    generator.add(Dense(units= 256, input_dim = 100))
    
    # 1번 층의 활성화 함수는 LeakeyReLU 사용 기울기 0.2로
    generator.add(LeakyReLU(0.2))
    
    # 2번 층
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    # 마지막 층(출력층)의 호라성화 함수는 tanh를 사용하고
    # 784개로 노드가 있는데 mnist 데이터 못브이 28x28 픽셀로 이루어져 잇기 때문이다.
    generator.add(Dense(units=784, activation='tanh'))
    return generator

g = create_generator()
g.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 784)               402192    
                                                                 
Total params: 559,632
Trainable params: 559,632
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 판별자 신경망 만들기

def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(units=512, input_dim = 784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    # 진짜인지 가짜인지 두 개중 하나로 구분하는 신경망이므로 오차값은 이항교차 엔트로피(binary_crossentropy)시영
    # GAM을 만들려면 이전보단 더 정교하게 옵티마이저를 사용해야한다. 따라서 Adam 학습속도는 0.0002로 베타 최적화 값을 0.5로 설정
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr = 0.0002, beta_1=0.5))
    
    return discriminator

d = create_discriminator()
d.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


C:\Users\Y\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
# GAN  생성 함수 만들기

# 생성자 신경망과 판별자 신경망을 적절히 학습시키기
def create_gan(discriminator, generator):
    # 판별자가 학습을 못하도록 막아줌
    discriminator.trainable = False
    
    # 적대적 생성 신경망인 gan에 입력할 데이터의 모습을 정하는 코드, 입력할 데이터 형태를 만들어 줌
    # 입력하는 값은 100개의 값으로 이루어진 데이터임
    gan_input = Input(shape=(100, ))
    
    # 생성자 신경망에게 바로 윗줄에서 작성한 픽셀 100개의 값과 데이터의 전체수(Input(shape=(100,)))만큼 데이터를 넣는다.
    # 이 픽셀 100개는 노이즈 값, 이렇게 데이터를 넣으면 생성자가 만든 새로운 그림들이 변수 x에 저장된다.
    x = generator(gan_input)
    
    # 적대적 생성 신경망 gan의 결괏값 데이터를 정의해주는 코드로
    # 이 결괏값 데이터는 바로 판별자가 생상자가 만든 그림(x)를 보고 판단한 결과이다.
    gan_output = discriminator(x)
    
    # 적대적 신경망 gan 모델 만들기
    # input은 생성자 신경망이 만든 그림(노이즈값), 출력값은 판별자 신경망이 판단한 결과이다.
    gan = Model(inputs = gan_input, outputs = gan_output)
    
    # 신경망의 오차를 줄이기 위한 방법
    # 출력ㄱ밧이 OX를 판단하는 것이므로 이진
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

gan = create_gan(d, g)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_1 (Sequential)   (None, 784)               559632    
                                                                 
 sequential_3 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 1,093,137
Trainable params: 559,632
Non-trainable params: 533,505
_________________________________________________________________


In [15]:
# 결과 확인 함수 만들기

def plot_generated_images(generator):
    # 생성자에 넣어줄 노이즈값을 만들고 넘파이 랜덤값 생성 라이브러리중 정규 분포 함수 사용
    # scale = 1은 -1에서 1 사이의 값을 생성한다는 뜻
    # arg3 : 노이즈 100개를 생성하며, 각 노이즈는 숫자 100개씩으로 구성되어 있다는 뜻
    # 함수 호출마다 100개의 그림을 그려달라고 요청
    noise = np.random.normal(loc=0, scale =1, size=[100,100])
    
    # generator도 신경망 모델이므로 predictㄱ ㅏ능
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,  28, 28)
    plt.figure(figsize=(10, 10)) # 그림의 크기를 10x10으로 정해줌
    
    for i in range(generated_images.shape[0]):
        plt.subplot(10, 10, i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off') # 그림 이름을 넣지 않는다.
    
    plt.tight_layout()

In [ ]:
# 적대적 생성 신경망 훈련시키기

batch_size = 128 # 한 번에 몇 개의 그림을 학습시킬지
epochs = 5000 # 몇 번 반복해서 학습시킬지

# 신경망을 학습시킬 차례로 반복문과 tqdm 라이브러리를 사용해 반복 학습을 진행
for e in tqdm(range(epochs)):
    
    # 생성자에게 줄 노이즈값을 만듦
    # 이때 균일한 값을 생성할 수 잇도록 넘파이의 랜덤값 생성 라이브러리 중 정규 분포 함수 사용
    # 평균을 0으로 -1, 1까지 128개(batch_size)만큼 만드는대 각각 100개씩
    noise = np.random.normal(0,1,[batch_size, 100])
    # 생성자 모델에 노이즈를 입력해, 생성자 신경망이 그림을 그린 뒤(예측한 뒤) 그림 결과를 generated_images에 저장
    generated_images = g.predict(noise)
    
    # 학습할 때마다 다양한 손글씨를 인공지능이 학습하도록
    # 실제 mnist 데이터셋 10000개에서 128개만 랜덤으로 뽑는 코드
    image_batch = mnist_data[np.random.randint(low = 0, hight = mnist_data.shape[0], size = batch_size)]
    
    # 실제 mnist데이터인 image_batch와 가짜 그림 generated_images를 합침(한줄로 세움)
    X = np.concatenate([image_batch, generated_images])
    
    y_dis = np.zeros(2*batch_size) 
    y_dis[:batch_size] = 1 # 앞의 128개는 진짜이므로 1이라고 true를 표기함
    
    # 정답을 보고 학습해 생성자가 만든 그림과 진짜 그림을 구분할 수 있어야 하기 때문에
    # 처음에는 판별자가 먼저 학습할 수 있도록 해야한다.
    d.trainable = True
    d.train_on_batch(X, y_dis) # 판별자를 학습시킴 -> 오차를 줄이는 방식으로 학습해감
    
    # 새로운 노이즈값 생성
    noise = np.random.normal(0,1, [batch_size, 100])
    
    # gan에 넣어줄 값을 만들고 모두 1로 설정
    y_gen = np.ones(batch_size)
    d.trainable = False # 판별자가 학습 못하게 설정(생성자가 만든 그림이 진짜인지 가짜인지 판별하는 역할만함)
    
    # gan에서 노이즈값을 입력으로하고 y_gen을 출력으로 넣음
    # 출력값은 입력된 그림이 진짜인지 가짜인지 알려주는 판별자 신경망을 거쳐서 나오는 값(판별 결과)이다.
    gan.train_on_batch(noise, y_gen)
    
    # 각 에포크별로 훈련을 잘하는 지 살펴보기
    # 첫 번째 에포크 e == 1과 1000, 2000 ... 5000번째 에포크일때 생성자가 만든 그림 출력
    if e == 1 or e%1000 == 0:
        plot_generated_images(g)
